In [3]:
import sys

'''
run like python unfold_fullstats.py Rapgap nominal 1
options for the sample are Rapgap and Django and options for the setting are nominal, sys_0, sys_1, ...
'''

#print("Running on MC sample",sys.argv[1],"with setting",sys.argv[2])

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
from matplotlib import gridspec
import time

from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler

import os
#os.environ['CUDA_VISIBLE_DEVICES']=sys.argv[3] #"1"

#physical_devices = tf.config.list_physical_devices('GPU') 
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

def weighted_binary_crossentropy(y_true, y_pred):
    weights = tf.gather(y_true, [1], axis=1) # event weights
    y_true = tf.gather(y_true, [0], axis=1) # actual y_true for loss

    # Clip the prediction value to prevent NaN's and Inf's
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    t_loss = -weights * ((y_true) * K.log(y_pred) +
                         (1 - y_true) * K.log(1 - y_pred))
    return K.mean(t_loss)

def reweight(events):
    f = model.predict(events, batch_size=5000)
    weights = f / (1. - f)
    return np.squeeze(np.nan_to_num(weights,posinf=1))


bins = {}

#y eSigma
bins[0] = np.logspace(np.log10(0.01),np.log10(1),9)
#electron eta
bins[1] = np.linspace(-1,2.5,6)



mcname = 'Rapgap'
tag = 'nominal'
mcname = mcname + "_" + tag


#logfile
logfile = open("log_files/"+mcname+".txt","w")

#Read in the data
data = pd.read_pickle("datafiles/data.pkl")

theta_unknown_S = data[['event_y_es','event_Q2_es','event_x_es',
                        'event_y_s','event_Q2_s','event_x_s',
                        'event_y_e', 'event_Q2_e','event_x_e',
                        'e_px','e_py','e_pz','HFS_px','HFS_py','HFS_pz','HFS_E','HFS_eta']].to_numpy()

#Read in the MC
mc = pd.read_pickle("datafiles/"+mcname+".pkl")
#mc = mc[:1000000]
print(len(mc))
theta0_S = mc[['event_y_es','event_Q2_es', 'event_x_es',
               'event_y_s','event_Q2_s', 'event_x_s',
               'event_y_e', 'event_Q2_e', 'event_x_e',
               'e_px','e_py','e_pz','HFS_px','HFS_py','HFS_pz','HFS_E','HFS_eta']].to_numpy()
theta0_G = mc[['gen_event_y_es','gen_event_Q2_es', 'gen_event_x_es',
               'gen_event_y_s','gen_event_Q2_s', 'gen_event_x_s',
               'gen_event_y_e', 'gen_event_Q2_e', 'gen_event_x_e',
               'gene_px','gene_py','gene_pz','genHFS_px','genHFS_py','genHFS_pz','genHFS_E','genHFS_eta']].to_numpy()
weights_MC_sim = mc['wgt']
pass_reco = np.array(mc['pass_reco'])
pass_truth = np.array(mc['pass_truth'])
pass_fiducial = np.array(mc['pass_fiducial'])

del mc
gc.collect()

#Early stopping
earlystopping = EarlyStopping(patience=10,
                              verbose=True,
                              restore_best_weights=True)

#Now, for the unfolding!

nepochs = 10000

starttime = time.time()

NNweights_step2 = np.ones(len(theta0_S))

#Set up the model
inputs = Input((17, ))
hidden_layer_1 = Dense(50, activation='relu')(inputs)
hidden_layer_2 = Dense(100, activation='relu')(hidden_layer_1)
hidden_layer_3 = Dense(50, activation='relu')(hidden_layer_2)
outputs = Dense(1, activation='sigmoid')(hidden_layer_3)
mymodel = Model(inputs=inputs, outputs=outputs)

61416831


In [ ]:
for iteration in range(5):

    #Process the data
    print("on iteration=",iteration," processing data for step 1, time elapsed=",time.time()-starttime)
    logfile.write("on iteration="+str(iteration)+" processing data for step 1, time elapsed="+str(time.time()-starttime)+"\n")

    xvals_1 = np.concatenate([theta0_S[pass_reco==1],theta_unknown_S])
    yvals_1 = np.concatenate([np.zeros(len(theta0_S[pass_reco==1])),np.ones(len(theta_unknown_S))])
    weights_1 = np.concatenate([NNweights_step2[pass_reco==1]*weights_MC_sim[pass_reco==1],np.ones(len(theta_unknown_S))])

    scaler_data = StandardScaler()
    scaler_data.fit(theta_unknown_S)
    xvals_1 = scaler_data.transform(xvals_1)

    X_train_1, X_test_1, Y_train_1, Y_test_1, w_train_1, w_test_1 = train_test_split(xvals_1, yvals_1, weights_1,test_size=0.5)
    del xvals_1,yvals_1,weights_1
    gc.collect()

    Y_train_1 = np.stack((Y_train_1, w_train_1), axis=1)
    Y_test_1 = np.stack((Y_test_1, w_test_1), axis=1)
    del w_train_1,w_test_1
    gc.collect()

    print("on iteration=",iteration," done processing data for step 1, time elapsed=",time.time()-starttime)
    print("data events = ",len(X_train_1[Y_train_1[:,0]==1]))
    print("MC events = ",len(X_train_1[Y_train_1[:,0]==0]))

    logfile.write("on iteration="+str(iteration)+" done processing data for step 1, time elapsed="+str(time.time()-starttime)+"\n")
    logfile.write("data events = "+str(len(X_train_1[Y_train_1[:,0]==1]))+"\n")
    logfile.write("MC events = "+str(len(X_train_1[Y_train_1[:,0]==0]))+"\n")
    
    #Step 1
    print("on step 1, time elapsed=",time.time()-starttime)
    logfile.write("on step 1, time elapsed="+str(time.time()-starttime)+"\n")
    
    opt = tf.keras.optimizers.Adam(learning_rate=2e-6)
    mymodel.compile(loss=weighted_binary_crossentropy,
                      optimizer=opt,
                      metrics=['accuracy'])

    hist_s1 =  mymodel.fit(X_train_1,Y_train_1,
              epochs=nepochs,
              batch_size=50000,
              validation_data=(X_test_1,Y_test_1),
              callbacks=[earlystopping],
              verbose=1)

    print("done with step 1, time elapsed=",time.time()-starttime)
    logfile.write("done with step 1, time elapsed="+str(time.time()-starttime)+"\n")
    
    #Now, let's do some checking.

    ###
    # Loss
    ###

    fig = plt.figure(figsize=(7, 5)) 
    gs = gridspec.GridSpec(1, 1, height_ratios=[1]) 
    ax0 = plt.subplot(gs[0])
    ax0.yaxis.set_ticks_position('both')
    ax0.xaxis.set_ticks_position('both')
    ax0.tick_params(direction="in",which="both")
    ax0.minorticks_on()
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)

    plt.plot(np.array(hist_s1.history['loss']),label="loss")
    plt.plot(np.array(hist_s1.history['val_loss']),label="val. loss",ls=":")
    plt.xlabel("Epoch",fontsize=20)
    plt.ylabel("Loss",fontsize=20)
    plt.title("OmniFold iteration "+str(iteration)+", step 1",loc="left",fontsize=20)
    plt.text(0.05, 1.15,'H1', horizontalalignment='center', verticalalignment='center', transform = ax0.transAxes, fontsize=25, fontweight='bold')
    plt.legend(frameon=False,fontsize=20)
    plt.locator_params(axis='x', nbins=5)
    plt.xscale("log")
    fig.savefig("storage_plots/"+mcname+"Iteration"+str(iteration)+"_Step1_loss.pdf",bbox_inches='tight')
    
     #save the model
    tf.keras.models.save_model(mymodel,"models/"+mcname+"_step1_iteration"+str(iteration))

    
    mypred = mymodel.predict(scaler_data.transform(np.nan_to_num(theta0_S,posinf=0,neginf=0)),batch_size=10000)
    mypred = mypred/(1.-mypred)
    mypred = mypred[:,0]
    mypred = np.squeeze(np.nan_to_num(mypred,posinf=1))
    
    
    ###
    # y
    ###

    fig = plt.figure(figsize=(7, 5)) 
    gs = gridspec.GridSpec(2, 1, height_ratios=[2,1]) 
    ax0 = plt.subplot(gs[0])
    ax0.yaxis.set_ticks_position('both')
    ax0.xaxis.set_ticks_position('both')
    ax0.tick_params(direction="in",which="both")
    ax0.minorticks_on()
    plt.xticks(fontsize=0)
    plt.yticks(fontsize=20)

    n_data,b,_=plt.hist(theta_unknown_S[:,0],bins=bins[0],density=True,alpha=0.5,label="Data")
    n_MC,_,_=plt.hist(theta0_S[pass_reco==1][:,0],bins=bins[0],weights=weights_MC_sim[pass_reco==1]*NNweights_step2[pass_reco==1],density=True,histtype="step",color="black",label="MC + step 2 (i-1)")
    n_Omni_step1_y,_,_=plt.hist(theta0_S[pass_reco==1][:,0],bins=bins[0],weights=weights_MC_sim[pass_reco==1]*NNweights_step2[pass_reco==1]*mypred[pass_reco==1],density=True,histtype="step",color="black",ls=":",label="MC + step 1 (i)")

    plt.ylabel("Normalized to unity",fontsize=20)
    plt.title("OmniFold iteration "+str(iteration)+", step 1",loc="left",fontsize=20)
    plt.text(0.05, 1.25,'H1', horizontalalignment='center', verticalalignment='center', transform = ax0.transAxes, fontsize=25, fontweight='bold')
    plt.legend(frameon=False,fontsize=15)
    plt.locator_params(axis='x', nbins=5)
    #plt.ylim([0,0.8])

    ax1 = plt.subplot(gs[1])
    ax1.yaxis.set_ticks_position('both')
    ax1.xaxis.set_ticks_position('both')
    ax1.tick_params(direction="in",which="both")
    ax1.minorticks_on()
    plt.yticks(fontsize=20)
    plt.xticks(fontsize=20)

    plt.xlabel("Detector-level $y$",fontsize=15)
    plt.ylabel("MC/data",fontsize=15)

    plt.plot(0.5*(b[0:-1]+b[1:]),n_MC/n_data,ls="--",color="black")
    plt.plot(0.5*(b[0:-1]+b[1:]),n_Omni_step1_y/n_data,ls=":",color="black")

    fig.savefig("storage_plots/"+mcname+"_Iteration"+str(iteration)+"_Step1_y.pdf",bbox_inches='tight')
    np.save("storage_files/"+mcname+"_n_Omni_step1_y_iteration"+str(iteration),n_Omni_step1_y)
    
    
    print("time for step 2, time elapsed=",time.time()-starttime)
    logfile.write("time for step 2, time elapsed="+str(time.time()-starttime)+"\n")
    
    xvals_2 = np.concatenate([theta0_G[pass_truth==1],theta0_G[pass_truth==1]])
    yvals_2 = np.concatenate([np.zeros(len(theta0_G[pass_truth==1])),np.ones(len(theta0_G[pass_truth==1]))])

    xvals_2 = scaler_data.transform(xvals_2)

    NNweights = mymodel.predict(scaler_data.transform(np.nan_to_num(theta0_S[pass_truth==1],posinf=0,neginf=0)),batch_size=10000)
    NNweights = NNweights/(1.-NNweights)
    NNweights = NNweights[:,0]
    NNweights = np.squeeze(np.nan_to_num(NNweights,posinf=1))
    NNweights[pass_reco[pass_truth==1]==0] = 1.
    weights_2 = np.concatenate([NNweights_step2[pass_truth==1]*weights_MC_sim[pass_truth==1],NNweights*NNweights_step2[pass_truth==1]*weights_MC_sim[pass_truth==1]])

    X_train_2, X_test_2, Y_train_2, Y_test_2, w_train_2, w_test_2 = train_test_split(xvals_2, yvals_2, weights_2,test_size=0.5)
    del xvals_2,yvals_2,weights_2
    gc.collect()

    Y_train_2 = np.stack((Y_train_2, w_train_2), axis=1)
    Y_test_2 = np.stack((Y_test_2, w_test_2), axis=1)
    del w_train_2,w_test_2
    gc.collect()

    print("on iteration=",iteration," done processing data for step 2, time elapsed=",time.time()-starttime)
    print("MC events = ",len(X_train_1[Y_train_1[:,0]==1]))
    print("MC events = ",len(X_train_1[Y_train_1[:,0]==0]))

    logfile.write("on iteration="+str(iteration)+" done processing data for step 2, time elapsed="+str(time.time()-starttime)+"\n")
    logfile.write("MC events = "+str(len(X_train_1[Y_train_1[:,0]==1]))+"\n")
    logfile.write("MC events = "+str(len(X_train_1[Y_train_1[:,0]==0]))+"\n")
    
    #step 2
    opt = tf.keras.optimizers.Adam(learning_rate=5e-6)
    mymodel.compile(loss=weighted_binary_crossentropy,
                      optimizer=opt,
                      metrics=['accuracy'])

    hist_s2 =  mymodel.fit(X_train_2,Y_train_2,
              epochs=nepochs,
              batch_size=100000,
              validation_data=(X_test_2,Y_test_2),
              callbacks=[earlystopping],
              verbose=1)

    print("on iteration=",iteration," finished step 2; time elapsed=",time.time()-starttime)
    logfile.write("on iteration="+str(iteration)+" finished step 2; time elapsed="+str(time.time()-starttime)+"\n")
    
    NNweights_step2_hold = mymodel.predict(scaler_data.transform(theta0_G),batch_size=10000)
    NNweights_step2_hold = NNweights_step2_hold/(1.-NNweights_step2_hold)
    NNweights_step2_hold = NNweights_step2_hold[:,0]
    NNweights_step2_hold = np.squeeze(np.nan_to_num(NNweights_step2_hold,posinf=1))
    NNweights_step2_hold[pass_truth==0] = 1.
    NNweights_step2 = NNweights_step2_hold*NNweights_step2

    #Now, let's do some checking.

    ###
    # Loss
    ###

    fig = plt.figure(figsize=(7, 5)) 
    gs = gridspec.GridSpec(1, 1, height_ratios=[1]) 
    ax0 = plt.subplot(gs[0])
    ax0.yaxis.set_ticks_position('both')
    ax0.xaxis.set_ticks_position('both')
    ax0.tick_params(direction="in",which="both")
    ax0.minorticks_on()
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)

    plt.plot(np.array(hist_s2.history['loss']),label="loss")
    plt.plot(np.array(hist_s2.history['val_loss']),label="val. loss",ls=":")
    plt.xlabel("Epoch",fontsize=20)
    plt.ylabel("Loss",fontsize=20)
    plt.title("OmniFold iteration "+str(iteration)+", step 2",loc="left",fontsize=20)
    plt.text(0.05, 1.15,'H1', horizontalalignment='center', verticalalignment='center', transform = ax0.transAxes, fontsize=25, fontweight='bold')
    plt.legend(frameon=False,fontsize=20)
    plt.locator_params(axis='x', nbins=5)
    plt.xscale("log")
    fig.savefig("storage_plots/"+mcname+"_Iteration"+str(iteration)+"_Step2_loss.pdf",bbox_inches='tight')
    
    #save the model
    tf.keras.models.save_model(mymodel,"models/Rapgap_nominal_step2_iteration"+str(iteration))

    ###
    # y
    ###

    fig = plt.figure(figsize=(7, 5)) 
    gs = gridspec.GridSpec(2, 1, height_ratios=[2,1]) 
    ax0 = plt.subplot(gs[0])
    ax0.yaxis.set_ticks_position('both')
    ax0.xaxis.set_ticks_position('both')
    ax0.tick_params(direction="in",which="both")
    ax0.minorticks_on()
    plt.xticks(fontsize=0)
    plt.yticks(fontsize=20)

    n_data,b,_=plt.hist(theta0_G[pass_fiducial==1][:,0],bins=bins[0],weights=weights_MC_sim[pass_fiducial==1]*NNweights[pass_fiducial==1]*NNweights_step2[pass_fiducial==1]/NNweights_step2_hold[pass_fiducial==1],density=True,alpha=0.5,label="MC + step 1")
    n_MC,_,_=plt.hist(theta0_G[pass_fiducial==1][:,0],bins=bins[0],weights=weights_MC_sim[pass_fiducial==1],density=True,histtype="step",color="black",label="MC")
    n_Omni_step2_y,_,_=plt.hist(theta0_G[pass_fiducial==1][:,0],bins=bins[0],weights=weights_MC_sim[pass_fiducial==1]*NNweights_step2[pass_fiducial==1],density=True,histtype="step",color="black",ls=":",label="MC + step 2")

    plt.ylabel("Normalized to unity",fontsize=20)
    plt.title("OmniFold iteration "+str(iteration)+", step 2",loc="left",fontsize=20)
    plt.text(0.05, 1.25,'H1', horizontalalignment='center', verticalalignment='center', transform = ax0.transAxes, fontsize=25, fontweight='bold')
    plt.legend(frameon=False,fontsize=15)
    plt.locator_params(axis='x', nbins=5)
    #plt.ylim([0,0.8])

    ax1 = plt.subplot(gs[1])
    ax1.yaxis.set_ticks_position('both')
    ax1.xaxis.set_ticks_position('both')
    ax1.tick_params(direction="in",which="both")
    ax1.minorticks_on()
    plt.yticks(fontsize=20)
    plt.xticks(fontsize=20)

    plt.xlabel("Particle-level $y$",fontsize=15)
    plt.ylabel("step 2/step 1",fontsize=15)

    plt.plot(0.5*(b[0:-1]+b[1:]),n_MC/n_data,ls="--",color="black")
    plt.plot(0.5*(b[0:-1]+b[1:]),n_Omni_step2_y/n_data,ls=":",color="black")

    fig.savefig("storage_plots/"+mcname+"_Iteration"+str(iteration)+"_Step2_y.pdf",bbox_inches='tight')
    np.save("storage_files/"+mcname+"_n_Omni_step2_y_iteration"+str(iteration),n_Omni_step2_y)
    

    print("done with the "+str(iteration)+"iteration, time elapsed=",time.time()-starttime)
    logfile.write("done with the "+str(iteration)+"iteration, time elapsed="+str(time.time()-starttime)+"\n")
    
    pass

#tf.keras.models.save_model(mymodel,"models/Rapgap_nominal")
#logfile.close()
    
    

on iteration= 0  processing data for step 1, time elapsed= 0.08926081657409668
on iteration= 0  done processing data for step 1, time elapsed= 5.902114152908325
data events =  172725
MC events =  8856302
on step 1, time elapsed= 6.306073904037476
Epoch 1/10000
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
181/181 [==============================] - 4s 18ms/step - loss: 0.0260 - accuracy: 0.2230 - val_loss: 0.0260 - val_accuracy: 0.2145
Epoch 2/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0260 - accuracy: 0.2075 - val_loss: 0.0260 - val_accuracy: 0.2006
Epoch 3/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0259 - accuracy: 0.1951 - val_loss: 0.0259 - val_accuracy: 0.1895
Epoch 4/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0259 - accuracy: 0.1852 - val_loss: 0.0259 - val_accuracy: 0.1806
Epoch 5/10000
181/181 [========

181/181 [==============================] - 3s 16ms/step - loss: 0.0254 - accuracy: 0.2095 - val_loss: 0.0255 - val_accuracy: 0.2075
Epoch 53/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0254 - accuracy: 0.2086 - val_loss: 0.0255 - val_accuracy: 0.2084
Epoch 54/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0254 - accuracy: 0.2091 - val_loss: 0.0255 - val_accuracy: 0.2090
Epoch 55/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0254 - accuracy: 0.2093 - val_loss: 0.0255 - val_accuracy: 0.2109
Epoch 56/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0254 - accuracy: 0.2117 - val_loss: 0.0255 - val_accuracy: 0.2117
Epoch 57/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0254 - accuracy: 0.2110 - val_loss: 0.0255 - val_accuracy: 0.2127
Epoch 58/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0254 - accuracy: 0.2138 - val_loss: 0.0255 - val_acc

Epoch 108/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0254 - accuracy: 0.2449 - val_loss: 0.0254 - val_accuracy: 0.2447
Epoch 109/10000
181/181 [==============================] - 3s 15ms/step - loss: 0.0254 - accuracy: 0.2456 - val_loss: 0.0254 - val_accuracy: 0.2433
Epoch 110/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0254 - accuracy: 0.2440 - val_loss: 0.0254 - val_accuracy: 0.2436
Epoch 111/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0254 - accuracy: 0.2439 - val_loss: 0.0254 - val_accuracy: 0.2462
Epoch 112/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0253 - accuracy: 0.2448 - val_loss: 0.0254 - val_accuracy: 0.2463
Epoch 113/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0253 - accuracy: 0.2456 - val_loss: 0.0254 - val_accuracy: 0.2465
Epoch 114/10000
181/181 [==============================] - 3s 15ms/step - loss: 0.0253 - accuracy: 0.2462 - val_

181/181 [==============================] - 3s 16ms/step - loss: 0.0253 - accuracy: 0.2584 - val_loss: 0.0254 - val_accuracy: 0.2582
Epoch 164/10000
181/181 [==============================] - 3s 15ms/step - loss: 0.0253 - accuracy: 0.2593 - val_loss: 0.0254 - val_accuracy: 0.2589
Epoch 165/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0253 - accuracy: 0.2591 - val_loss: 0.0254 - val_accuracy: 0.2588
Epoch 166/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0253 - accuracy: 0.2596 - val_loss: 0.0254 - val_accuracy: 0.2589
Epoch 167/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0253 - accuracy: 0.2594 - val_loss: 0.0254 - val_accuracy: 0.2592
Epoch 168/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0253 - accuracy: 0.2592 - val_loss: 0.0254 - val_accuracy: 0.2610
Epoch 169/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0253 - accuracy: 0.2612 - val_loss: 0.0254 - v

181/181 [==============================] - 3s 16ms/step - loss: 0.0253 - accuracy: 0.2674 - val_loss: 0.0254 - val_accuracy: 0.2668
Epoch 219/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0253 - accuracy: 0.2657 - val_loss: 0.0254 - val_accuracy: 0.2679
Epoch 220/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0253 - accuracy: 0.2683 - val_loss: 0.0254 - val_accuracy: 0.2665
Epoch 221/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0253 - accuracy: 0.2660 - val_loss: 0.0254 - val_accuracy: 0.2666
Epoch 222/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0253 - accuracy: 0.2682 - val_loss: 0.0254 - val_accuracy: 0.2665
Epoch 223/10000
181/181 [==============================] - 3s 15ms/step - loss: 0.0253 - accuracy: 0.2667 - val_loss: 0.0254 - val_accuracy: 0.2680
Epoch 224/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0253 - accuracy: 0.2679 - val_loss: 0.0254 - v

...

181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2804 - val_loss: 0.0253 - val_accuracy: 0.2800
Epoch 1759/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2798 - val_loss: 0.0253 - val_accuracy: 0.2792
Epoch 1760/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2800 - val_loss: 0.0253 - val_accuracy: 0.2786
Epoch 1761/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2790 - val_loss: 0.0253 - val_accuracy: 0.2809
Epoch 1762/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2792 - val_loss: 0.0253 - val_accuracy: 0.2801
Epoch 1763/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2795 - val_loss: 0.0253 - val_accuracy: 0.2805
Epoch 1764/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2795 - val_loss: 0.02

181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2794 - val_loss: 0.0253 - val_accuracy: 0.2811
Epoch 1814/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2810 - val_loss: 0.0253 - val_accuracy: 0.2794
Epoch 1815/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2787 - val_loss: 0.0253 - val_accuracy: 0.2804
Epoch 1816/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2801 - val_loss: 0.0253 - val_accuracy: 0.2794
Epoch 1817/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2805 - val_loss: 0.0253 - val_accuracy: 0.2791
Epoch 1818/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2792 - val_loss: 0.0253 - val_accuracy: 0.2795
Epoch 1819/10000
181/181 [==============================] - 3s 15ms/step - loss: 0.0252 - accuracy: 0.2796 - val_loss: 0.02

181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2803 - val_loss: 0.0253 - val_accuracy: 0.2796
Epoch 1869/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2798 - val_loss: 0.0253 - val_accuracy: 0.2796
Epoch 1870/10000
181/181 [==============================] - 3s 15ms/step - loss: 0.0252 - accuracy: 0.2797 - val_loss: 0.0253 - val_accuracy: 0.2801
Epoch 1871/10000
181/181 [==============================] - 3s 15ms/step - loss: 0.0252 - accuracy: 0.2805 - val_loss: 0.0253 - val_accuracy: 0.2788
Epoch 1872/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2805 - val_loss: 0.0253 - val_accuracy: 0.2782
Epoch 1873/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2791 - val_loss: 0.0253 - val_accuracy: 0.2794
Epoch 1874/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2796 - val_loss: 0.02

181/181 [==============================] - 3s 15ms/step - loss: 0.0252 - accuracy: 0.2799 - val_loss: 0.0253 - val_accuracy: 0.2803
Epoch 1924/10000
181/181 [==============================] - 3s 15ms/step - loss: 0.0252 - accuracy: 0.2801 - val_loss: 0.0253 - val_accuracy: 0.2802
Epoch 1925/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2806 - val_loss: 0.0253 - val_accuracy: 0.2795
Epoch 1926/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2800 - val_loss: 0.0253 - val_accuracy: 0.2802
Epoch 1927/10000
181/181 [==============================] - 3s 15ms/step - loss: 0.0252 - accuracy: 0.2797 - val_loss: 0.0253 - val_accuracy: 0.2799
Epoch 1928/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2798 - val_loss: 0.0253 - val_accuracy: 0.2797
Epoch 1929/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2798 - val_loss: 0.02

181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2806 - val_loss: 0.0253 - val_accuracy: 0.2809
Epoch 1979/10000
181/181 [==============================] - 3s 15ms/step - loss: 0.0252 - accuracy: 0.2808 - val_loss: 0.0253 - val_accuracy: 0.2809
Epoch 1980/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2806 - val_loss: 0.0253 - val_accuracy: 0.2795
Epoch 1981/10000
181/181 [==============================] - 3s 15ms/step - loss: 0.0252 - accuracy: 0.2798 - val_loss: 0.0253 - val_accuracy: 0.2794
Epoch 1982/10000
181/181 [==============================] - 3s 15ms/step - loss: 0.0252 - accuracy: 0.2795 - val_loss: 0.0253 - val_accuracy: 0.2813
Epoch 1983/10000
181/181 [==============================] - 3s 16ms/step - loss: 0.0252 - accuracy: 0.2801 - val_loss: 0.0253 - val_accuracy: 0.2810
Epoch 1984/10000
181/181 [==============================] - 3s 15ms/step - loss: 0.0252 - accuracy: 0.2796 - val_loss: 0.02

In [ ]:
#tf.keras.models.save_model(mymodel,"models/Rapgap_nominal")
#logfile.close()
    